In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Can increase for longer reasoning traces
lora_rank = 16 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Instruct-2507",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.7: Fast Qwen3 patching. Transformers: 4.55.2.
   \\   /|    NVIDIA GeForce RTX 4070 Ti SUPER. Num GPUs = 1. Max memory: 15.992 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth 2025.8.7 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [2]:
new_chat_template = \
    "{% for message in messages %}"\
        "{{'<|im_start|>' + message['role'] + '\\n' + message['content'] + '<|im_end|>' + '\\n'}}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}"\
        "{{ '<|im_start|>assistant\\n' }}"\
    "{% endif %}"
tokenizer.chat_template = new_chat_template

tokenizer.apply_chat_template([
        {
            "role": "user", 
            "content": "Write a satirical headline in the style of Onion News. Only write the headline."
        },
        {
            "role": "assistant",
            "content": "Relaxed Marie Kondo Now Says She Perfectly Happy Living In Waist-High Sewage"
        },
    ], tokenize=False)

'<|im_start|>user\nWrite a satirical headline in the style of Onion News. Only write the headline.<|im_end|>\n<|im_start|>assistant\nRelaxed Marie Kondo Now Says She Perfectly Happy Living In Waist-High Sewage<|im_end|>\n'

In [3]:
import random

from datasets import load_dataset, load_from_disk
import spacy

def get_as_messages(data):
    headline = get_headline(data['text'])
    return {
        'messages': [
            {'role': 'user', 'content': get_prompt(headline)},
            {'role': 'assistant', 'content': headline},
        ]
    }


SEP = '#~#'
def get_headline(text):
    return text.split(SEP)[0].strip()


def get_prompt(headline):
    if random.random() < 0.1:
        return get_prompt_no_topic()
    topics = extract_topics(headline)
    if not topics:
        return get_prompt_no_topic()
    else:
        return get_prompt_topics(topics)


nlp = spacy.load('en_core_web_sm')
def extract_topics(text, max_topics=2):
    doc = nlp(text.lower()) # lower casing the text seems to result in better parsing
    noun_chunks = [chunk for chunk in doc.noun_chunks if chunk.root.pos_ != 'PRON'] # get all noun chunks, ignoring pronouns
    noun_chunks.sort(key=key, reverse=True)
    return [chunk.text.strip() for chunk in noun_chunks[:max_topics]]


def key(span):
    # Returns the importance of a span. A span is important if it contains an entity or is long.
    entity_types = ['PERSON', 'ORG', 'GPE']
    is_entity = any(token.ent_type_ in entity_types for token in span)
    length = len(span.text.strip())
    return (is_entity, length)


def get_prompt_topics(topics):
    template = random.choice([
        "Write a satirical headline about {topic}.",
        "Create an Onion-style headline related to {topic}.",
        "Give me your funniest, most satirical take on {topic}.",
        "Satirize the topic in {topic} in a news headline.",
        "What would a satirical news headline about {topic} sound like?",
    ])
    return template.format(topic=' and '.join(topics))


def get_prompt_no_topic():
    return random.choice([
        'Write a satirical headline in the style of Onion News.',
        'Generate an original satirical news headline.',
        "Give me a headline that sounds like it's from the Onion.",
        'Come up with a funny, fake news headline.',
        'Write a headline that is both absurd and oddly believable.',
    ])

def apply_chat_template(data):
    messages = data['messages']
    chat = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt=False)
    return {'text': chat}
    

filepath = 'Onion-News-Guided-Prompts-Tokens'
try:
    ds = load_from_disk(filepath)
except:
    ds = load_dataset("Biddls/Onion_News")
    ds = ds.map(get_as_messages, remove_columns=['text'])
    ds = ds.map(apply_chat_template, remove_columns=['messages'])
    ds.save_to_disk(filepath)
ds['train'][:5]

{'text': ['<|im_start|>user\nGive me your funniest, most satirical take on relaxed marie kondo and waist-high sewage.<|im_end|>\n<|im_start|>assistant\nRelaxed Marie Kondo Now Says She Perfectly Happy Living In Waist-High Sewage<|im_end|>\n',
  '<|im_start|>user\nGive me your funniest, most satirical take on u.s. officials and correct amount.<|im_end|>\n<|im_start|>assistant\nU.S. Officials Call For Correct Amount Of Violence<|im_end|>\n',
  '<|im_start|>user\nGive me your funniest, most satirical take on kamala harris and communications assistant.<|im_end|>\n<|im_start|>assistant\nKamala Harris Asks Communications Assistant If She Can Take Them Out For Coffee And Pick Their Brain Sometime<|im_end|>\n',
  '<|im_start|>user\nSatirize the topic in fake nursing school diploma scheme in a news headline.<|im_end|>\n<|im_start|>assistant\n25 Arrested In Fake Nursing School Diploma Scheme<|im_end|>\n',
  '<|im_start|>user\nCome up with a funny, fake news headline.<|im_end|>\n<|im_start|>assis

In [4]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = ds['train'],
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 1, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        learning_rate = 2e-5, # Reduce to 2e-5 for long training runs
        logging_steps = 10,
        optim = "adamw_8bit",
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
        
        output_dir='./qwen3_lora_finetuned_v3',
        save_steps=500,
    ),
)

In [5]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

In [6]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 33,880 | Num Epochs = 1 | Total steps = 4,235
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 33,030,144 of 4,055,498,240 (0.81% trained)


Step,Training Loss
10,4.844400
20,2.336800
30,2.146900
40,2.133800
50,2.118800
60,1.933300
70,2.123000
80,2.156500
90,2.014900
100,2.261900


KeyboardInterrupt: 